In [1]:
%load_ext autoreload
%autoreload 2

In [42]:
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from tqdm import tqdm
import matplotlib as mpl
import numpy as np
import napari
from glasbey import create_palette

spots_directory = Path(r"D:\Tracking\DrosophilaNucleusTracking\data\spots")
save_path = Path(r"D:\Tracking\DrosophilaNucleusTracking\figures\output")

k = 1

files = list(spots_directory.glob("*_spots.csv"))

spots_df = pd.read_csv(list(spots_directory.glob("*_spots.csv"))[k])
stem = files[k].stem

pal = create_palette(100, lightness_bounds=(40, 95))

In [43]:
viewer = napari.Viewer(ndisplay=3)

X = spots_df[["frame", "display_x", "display_y", "display_z"]].values
X[:, 1] = X[:, 1]*3
X[:, 1:] = X[:, 1:]*2
color = [pal[int(tid) % 100] for tid in spots_df["track_id"]]
size = spots_df["radius"] * 4

from napari.utils.theme import get_theme, register_theme
custom_theme = get_theme("dark")
custom_theme.canvas = "#FFFFFF"
custom_theme.id = "glasbey"
register_theme("glasbey", custom_theme, "custom")
viewer.theme = "glasbey"

points = viewer.add_points(
    X, face_color=color, size=size, name=f"spots_{k}", border_width=1.0, border_width_is_relative=False,
    antialiasing=0.0, border_color="#000000",
)

viewer.camera.angles = (0, 180, 90)
viewer.camera.zoom = 0.60

glasbey_save_path = save_path / "glasbey" / stem
glasbey_save_path.mkdir(parents=True, exist_ok=True)

for frame in tqdm(spots_df["frame"].unique()):
    viewer.dims.set_point(0, frame)
    viewer.screenshot(glasbey_save_path / f"{frame:04d}.tif", canvas_only=True, scale=2, flash=False)


100%|██████████| 248/248 [00:21<00:00, 11.30it/s]
